## Menu Q&A Chatbot — Colab Demo

This notebook demonstrates a **hybrid** menu Q&A chatbot:

- **LLM for routing (optional)**: intent + entity extraction
- **Deterministic tools for facts (required)**: prices/calories/categories/discounts are computed from structured data (no hallucinated prices)

### Works without an OpenAI key
If `OPENAI_API_KEY` is **not** set, the system automatically uses the deterministic fallback router.

### Optional OpenAI routing
If you set `OPENAI_API_KEY`, the router will try the LLM path first and fall back automatically on any failure.


In [ ]:
# Section 1 — Setup (clone + install)
#
# In Colab, edit these two values to point at your repo.
GITHUB_REPO = "https://github.com/andres-troiano/menu-qa-chatbot.git"
REPO_DIR = "menu-qa-chatbot"

!git clone $GITHUB_REPO
%cd $REPO_DIR

# Install deps into the *current notebook Python* (so imports work).
!pip -q install -U uv
!uv pip install --system -e .


In [ ]:
# Section 2 — Load the menu index (Bootstrap)
from src.bootstrap import load_index_with_summary

index, summary = load_index_with_summary("data/dataset.json")
summary

## Section 3 — Run demo questions (NO API key required)

If `OPENAI_API_KEY` is not set, the router will automatically use the deterministic fallback router.


In [ ]:
from src.chat import answer

questions = [
    # Price Lookup
    "What is the price of a small NUTTY BOWL?",
    "How much is the ACAI ELIXIR?",
    "How much is the GREEN BOWL Large?",

    # Nutrition Lookup
    "How many calories does the GO GREEN smoothie have?",
    "Calories for Dragon Smoothie",

    # Category Listing
    "Which salads do you have?",
    "What bowls are available?",
    "Show me smoothies",

    # Discount Listing
    "What discounts are available?",
    "Which discounts include coupons?",

    # Discount Trigger Query
    "What items trigger BOGO Any Smoothie discount?",

    # Cross-Channel Price Comparison
    "Is the price for Smoothie - ACAI ELIXIR the same in all channels?",

]

for q in questions:
    print("Q:", q)
    print("A:", answer(q, index))
    print("-" * 80)


## Section 4 — Optional: Enable OpenAI routing

Optional. If you don’t have an OpenAI key, skip this section.

Set your key like this:


In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "PASTE_YOUR_KEY_HERE"

# Optional: override router model
# os.environ["OPENAI_MODEL"] = "gpt-4o-mini"


## Section 5 — Confirm router mode (debug)

You should see `meta.router` as:
- `"fallback"` if no key is set
- `"llm"` if a key is set and the LLM path succeeds


In [ ]:
from src.router import route

rr = route("What is the price of a small NUTTY BOWL?")
rr

## Section 6 — LLM-powered routing examples (optional)

If you enabled `OPENAI_API_KEY`, these queries will typically route via the LLM.

Important behavior:
- If the dataset does not contain channel pricing overrides, the answer should be honest (not invented).
- If discount triggers can’t be fully joined, the answer should be best-effort + explain the limitation.


In [ ]:
questions = [
    "How much is the nutty bowl if I get the small one?",
    "Do you have any salads?",
    "Is the price for Smoothie - ACAI ELIXIR the same in all channels?",
    "What items trigger a BOGO Any Smoothie discount?",
]

for q in questions:
    rr = route(q)
    print(q, "=>", rr.meta)
    
    print("Q:", q)
    print("A:", answer(q, index))
    print("-" * 80)


## Section 7 — Optional: Interactive mini chat

Run this cell to ask questions interactively.


In [ ]:
while True:
    q = input("Ask a menu question (or 'exit'): ").strip()
    if q.lower() in {"exit", "quit"}:
        break
    print(answer(q, index))
